In [1]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# this is the EM specific package for querying the EM data
from caveclient import CAVEclient


from meshparty import trimesh_io, trimesh_vtk
from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

query timestamp: 2022-09-22 00:00:00


In [2]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)
# # # if you have not yet setup this computer, uncomment this below line
# # # paste the token from the website in, and run the line


In [3]:
# client.auth.save_token(token="6c63712320bb5b3968adabc0d60c635d", overwrite=True)

# # # then comment or delete the line as you don't need to run it on this computer  again

In [4]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [5]:
t1_mns_df = client.materialize.query_table('motor_neuron_table_v7')

In [6]:
t1_mns_df

,id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position
0,290,t,A3_T1L,trochanter_flexor_in_coxa_Ac,72764168838880663,648518346479846132,"[21575, 87200, 2242]"
1,291,t,D1_T1L,tergopleural_promotor_pleural_promotor_in_thor...,72975343790906622,648518346491655185,"[28385, 88521, 2251]"
2,283,t,A4_T1L,pleural_remotor_and_abductor_in_thorax_miller_29,72834537583100984,648518346498402097,"[24350, 87262, 2285]"
3,292,t,D1_T1L,tergopleural_promotor_pleural_promotor_in_thor...,72975275004512296,648518346520551990,"[26759, 87253, 2174]"
4,285,t,A4_T1L,sternal_adductor_miller_in_thorax_miller_33,72904975181026144,648518346486809241,"[26181, 89068, 2521]"
...,...,...,...,...,...,...,...
134,311,t,L1_T1L,main_tibia_flexor_in_femur_4,72623500137174407,648518346480295739,"[18229, 89516, 2400]"
135,300,t,L1_T1L,auxiliary_tibia_flexor_in_femur_A2,72975275272752584,648518346482850213,"[28646, 87284, 2618]"
136,346,t,V4_T1L,sternal_anterior_rotator_in_thorax_miller_31,72623499935890376,648518346483739567,"[17144, 89099, 2005]"
137,341,t,V2_T1L,tergotrochanter_in_thorax_1,72834674753554975,648518346493010864,"[22701, 91045, 1692]"


In [7]:
side = []
nerve = []
segment = []
function = []
muscle = []
rank = []
segID = []

# Iterate down the rows and find the segment, muscle, function rank from the cell_type name in the data frame
for index, row in t1_mns_df.iterrows():
    clss = row.classification_system
    nerve = nerve + [utils.xnerve(clss)]
    if 'R' in clss:
        side = side + ['R']
    elif 'L' in clss:
        side = side + ['L']
    
    ct = row.cell_type
    
    seg,rnk = utils.xsegment(ct)
    segment = segment+[seg]
    rank = rank+[rnk]
    
    mscl = utils.xmuscle(ct)
    muscle = muscle+[mscl]
    
    fcn = utils.xfunction(ct)    
    function = function+[fcn];

    segID = segID+[row.pt_root_id]
    
# Standard way of creating a multiindex object
# https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html
arrays = [
    side,
    nerve,
    segment,
    function,
    muscle,
    rank,
    segID,
            ]
mn_tuples = list(zip(*arrays))
mn_index = pd.MultiIndex.from_tuples(mn_tuples, names=['side','nerve',"segment", 'function','muscle','rank','segID'])

In [8]:
t1_mns_df.index = mn_index
t1_mns_df

id  \
side nerve     segment function muscle                                             rank segID                     
L    Accessory coxa    flex     trochanter_flexor                                  Ac   648518346479846132  290   
     Dorsal    thorax  stance   tergopleural_promotor_pleural_promotor_miller_2... 0    648518346491655185  291   
     Accessory thorax  swing    pleural_remotor_and_abductor_miller_29             0    648518346498402097  283   
     Dorsal    thorax  stance   tergopleural_promotor_pleural_promotor_miller_2... 0    648518346520551990  292   
     Accessory thorax  stance   sternal_adductor_miller_miller_in_thorax_miller_33 0    648518346486809241  285   
...                                                                                                         ...   
     Leg       femur   flex     main_tibia_flexor                                  4    648518346480295739  311   
                                auxiliary_tibia_flexor                             A2   648518346482850213  300   
     Ventral   thorax  stance   sternal_anterior_rotator_miller_31                 0    648518346483739567  346   
                       extend   tergotrochanter                                    1    648518346493010864  341   
                                                                                   3    648518346474565314  343   

                                                                                                           valid  \
side nerve     segment function muscle                                             rank segID                      
L    Accessory coxa    flex     trochanter_flexor                                  Ac   648518346479846132     t   
     Dorsal    thorax  stance   tergopleural_promotor_pleural_promotor_miller_2... 0    648518346491655185     t   
     Accessory thorax  swing    pleural_remotor_and_abductor_miller_29             0    648518346498402097     t   
     Dorsal    thorax  stance   tergopleural_promotor_pleural_promotor_miller_2... 0    648518346520551990     t   
     Accessory thorax  stance   sternal_adductor_miller_miller_in_thorax_miller_33 0    648518346486809241     t   
...                                                                                                          ...   
     Leg       femur   flex     main_tibia_flexor                                  4    648518346480295739     t   
                                auxiliary_tibia_flexor                             A2   648518346482850213     t   
     Ventral   thorax  stance   sternal_anterior_rotator_miller_31                 0    648518346483739567     t   
                       extend   tergotrochanter                                    1    648518346493010864     t   
                                                                                   3    648518346474565314     t   

                                                                                                           classification_system  \
side nerve     segment function muscle                                             rank segID                                      
L    Accessory coxa    flex     trochanter_flexor                                  Ac   648518346479846132                A3_T1L   
     Dorsal    thorax  stance   tergopleural_promotor_pleural_promotor_miller_2... 0    648518346491655185                D1_T1L   
     Accessory thorax  swing    pleural_remotor_and_abductor_miller_29             0    648518346498402097                A4_T1L   
     Dorsal    thorax  stance   tergopleural_promotor_pleural_promotor_miller_2... 0    648518346520551990                D1_T1L   
     Accessory thorax  stance   sternal_adductor_miller_miller_in_thorax_miller_33 0    648518346486809241                A4_T1L   
...                                                                                                                          ...   
     Leg       femur   flex     main_tibia_flexo

In [9]:
muscle_tuple_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',
    # 'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]

for key in pool_keys:
    mn_tup = muscle_tuple_dict[key]
    t1_mns_df.loc[mn_tup,'preferred_pool'] = key

t1_mns_df.preferred_pool = t1_mns_df.preferred_pool.astype("category")
t1_mns_df.preferred_pool = t1_mns_df.preferred_pool.cat.set_categories(pool_keys)

t1_mns_df.sort_values(["preferred_pool"])


id  \
side nerve     segment    function muscle                                 rank   segID                     
L    Accessory thorax     swing    pleural_remotor_and_abductor_miller_29 0      648518346486098834  284   
                                   sternal_posterior_rotator_miller_32    0      648518346480104102  281   
                                   pleural_remotor_and_abductor_miller_29 0      648518346498402097  283   
                                   sternal_posterior_rotator_miller_32    0      648518346499790035  280   
                                                                                 648518346494806962  279   
...                                                                                                  ...   
R    Ventral   thorax     stance   sternal_anterior_rotator_miller_31     0      648518346495499883  417   
     Leg       tibia      flex     tarsus_depressor                       medial 648518346520229713  390   
               trochanter reductor femur_reductor                         0      648518346493764856  399   
               femur      flex     auxiliary_tibia_flexor                 A2     648518346465963568  368   
               ltm        claw     ltm                                    large  648518346503947872  381   

                                                                                                    valid  \
side nerve     segment    function muscle                                 rank   segID                      
L    Accessory thorax     swing    pleural_remotor_and_abductor_miller_29 0      648518346486098834     t   
                                   sternal_posterior_rotator_miller_32    0      648518346480104102     t   
                                   pleural_remotor_and_abductor_miller_29 0      648518346498402097     t   
                                   sternal_posterior_rotator_miller_32    0      648518346499790035     t   
                                                                                 648518346494806962     t   
...                                                                                                   ...   
R    Ventral   thorax     stance   sternal_anterior_rotator_miller_31     0      648518346495499883     t   
     Leg       tibia      flex     tarsus_depressor                       medial 648518346520229713     t   
               trochanter reductor femur_reductor                         0      648518346493764856     t   
               femur      flex     auxiliary_tibia_flexor                 A2     648518346465963568     t   
               ltm        claw     ltm                                    large  648518346503947872     t   

                                                                                                    classification_system  \
side nerve     segment    function muscle                                 rank   segID                                      
L    Accessory thorax     swing    pleural_remotor_and_abductor_miller_29 0      648518346486098834                A4_T1L   
                                   sternal_posterior_rotator_miller_32    0      648518346480104102                A1_T1L   
                                   pleural_remotor_and_abductor_miller_29 0      648518346498402097                A4_T1L   
                                   sternal_posterior_rotator_miller_32    0      648518346499790035                A1_T1L   
                                                                                 648518346494806962                A1_T1L   
...                                                                                                                   ...   
R    Ventral   thorax     stance   sternal_anterior_rotator_miller_31     0      648518346495499883                V4_T1R   
     Leg       tibia      flex     tarsus_depressor                       medial 648518346520229713                L1_T1R   
               trochanter red

In [ ]:
utils.save_df_as_pickle(t1_mns_df,'motor_neuron_table_v7_with_pool')